<a href="https://colab.research.google.com/github/jajapuramshivasai/Doubly-Stochastic-Transformers/blob/main/skyrmionic_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Micromagnetics and Skyrmion Dynamics: Theoretical Framework

This notebook simulates the dynamics of magnetic spin textures—specifically Ferromagnetic (FM) and Antiferromagnetic (AFM) skyrmions—using the continuum micromagnetic framework. Instead of modeling individual discrete atoms, we approximate the magnetic state as a continuous vector field: the normalized magnetization $\mathbf{m}(\mathbf{r}, t)$, where $|\mathbf{m}| = 1$.

## 1. The Energy Functional and Effective Field

The state of a magnetic system is governed by its free energy $E_\text{total}$. The system constantly seeks to minimize this energy. At any point in time, the local spins experience an **Effective Magnetic Field** ($\mathbf{H}_\text{eff}$), which is defined as the negative variational derivative of the energy functional with respect to the magnetization:

$$\mathbf{H}_\text{eff} = -\frac{1}{\mu_0 M_s} \frac{\delta E_\text{total}}{\delta \mathbf{m}}$$

Where $\mu_0$ is the vacuum permeability and $M_s$ is the saturation magnetization. In our numerical model, $E_\text{total}$ consists of several competing fundamental interactions:

### A. Exchange Interaction ($E_\text{ex}$)
The quantum mechanical Heisenberg exchange interaction dictates that neighboring spins want to align parallel (in ferromagnets) or antiparallel (in antiferromagnets). It heavily penalizes sharp spatial variations in the magnetization.
$$\mathbf{H}_\text{ex} = \frac{2A}{\mu_0 M_s} \nabla^2 \mathbf{m}$$
Where $A$ is the exchange stiffness constant.

### B. Dzyaloshinskii-Moriya Interaction ($E_\text{DMI}$)
This antisymmetric exchange arises in materials lacking structural inversion symmetry (like interfaces between heavy metals and ferromagnets). Unlike standard exchange, DMI prefers neighboring spins to sit at **90-degree angles**, causing the continuous twisting required to form a skyrmion. For interfacial DMI:
$$\mathbf{H}_\text{DMI} = \frac{2D}{\mu_0 M_s} \left[ (\nabla \cdot \mathbf{m})\hat{z} - \nabla m_z \right]$$
Where $D$ is the DMI constant.

### C. Magnetic Anisotropy ($E_\text{ani}$)
Perpendicular Magnetic Anisotropy (PMA) occurs due to spin-orbit coupling at crystalline interfaces, forcing the preferred state of the spins to point along an "easy axis" (here, the z-axis).
$$\mathbf{H}_\text{ani} = \frac{2K}{\mu_0 M_s} m_z \hat{z}$$
Where $K$ is the uniaxial anisotropy constant.

### D. Zeeman Energy ($E_\text{zee}$)
The energy contribution from an external, applied magnetic field $\mathbf{B}_\text{ext}$. It forces spins to align with the applied field.
$$\mathbf{H}_\text{zee} = \mathbf{B}_\text{ext} / \mu_0$$

---

## 2. Magnetization Dynamics: The LLGS Equation



The time evolution of the magnetization vector $\mathbf{m}$ is described by the **Landau-Lifshitz-Gilbert-Slonczewski (LLGS)** equation. The standard implicit form is:

$$\frac{\partial \mathbf{m}}{\partial t} = -\gamma \mathbf{m} \times \mathbf{H}_\text{eff} + \alpha \mathbf{m} \times \frac{\partial \mathbf{m}}{\partial t} + \mathbf{\tau}_\text{driver}$$

* **Precession ($-\gamma \mathbf{m} \times \mathbf{H}_\text{eff}$):** The gyroscopic motion of the spin around the local effective field ($\gamma$ is the gyromagnetic ratio).
* **Damping ($\alpha \mathbf{m} \times \frac{\partial \mathbf{m}}{\partial t}$):** The phenomenological Gilbert damping parameter ($\alpha$) causes the precessing spin to gradually spiral inward and align with the effective field, dissipating energy as heat (magnons/phonons).
* **Current-Induced Torque ($\mathbf{\tau}_\text{driver}$):** The Slonczewski spin-transfer torque or spin-orbit torque driven by an electrical current. For a spin current with polarization vector $\mathbf{p}$, the damping-like torque is approximated as: $\tau \propto \mathbf{m} \times (\mathbf{m} \times \mathbf{p})$.

To solve this numerically using explicit time-integration techniques (like 4th-Order Runge-Kutta), the equation is algebraically rearranged into its **explicit form**:

$$\frac{\partial \mathbf{m}}{\partial t} = -\frac{\gamma}{1+\alpha^2} \mathbf{m} \times \mathbf{H}_\text{eff} - \frac{\gamma \alpha}{1+\alpha^2} \mathbf{m} \times (\mathbf{m} \times \mathbf{H}_\text{eff}) + \mathbf{\tau}_\text{driver}$$

---

## 3. Topology and the Skyrmion Charge



A magnetic skyrmion is a topologically protected quasiparticle. This means it cannot be continuously deformed into the uniform ferromagnetic state without tearing the magnetic fabric (requiring a massive energy spike). We quantify this using the topological charge or winding number $Q$:

$$Q = \frac{1}{4\pi} \iint \mathbf{m} \cdot \left( \frac{\partial \mathbf{m}}{\partial x} \times \frac{\partial \mathbf{m}}{\partial y} \right) dx dy$$

For a perfect single skyrmion, $Q = \pm 1$. The background ferromagnetic state has $Q = 0$.

---

## 4. Antiferromagnetic (AFM) Skyrmions

Ferromagnetic skyrmions suffer from the **Skyrmion Hall Effect (SkHE)**. Because they carry a non-zero topological charge ($Q \neq 0$), moving them with an applied current generates a fictitious Magnus force, causing the skyrmion to move diagonally and crash into the edges of the racetrack.

Antiferromagnetic skyrmions solve this by utilizing two closely coupled magnetic sublattices ($\mathbf{m}_1$ and $\mathbf{m}_2$) that are forced into anti-alignment by a massive negative exchange coupling ($J_\text{AFM}$):

$$E_\text{AFM} = -J_\text{AFM} (\mathbf{m}_1 \cdot \mathbf{m}_2)$$

Because $\mathbf{m}_2 = -\mathbf{m}_1$, the topological charges cancel out ($Q_\text{total} = Q_1 + Q_2 = -1 + 1 = 0$). With zero net topological charge, the Magnus forces cancel perfectly, and the AFM skyrmion is propelled strictly longitudinally down the racetrack by the applied current. To track AFM dynamics, we observe the **Néel vector**:

$$\mathbf{L} = \frac{\mathbf{m}_1 - \mathbf{m}_2}{2}$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import matplotlib.cm as cm
from tqdm.notebook import tqdm # Import tqdm for progress bar

# ==========================================
# 1. Physics Parameters (AFM Racetrack)
# ==========================================
Nx, Ny = 50, 50      # Racetrack geometry
dx = 1.0
dt = 0.005           # Tiny dt for AFM THz dynamics
alpha = 0.2
gamma = 1.0

A_ex = 1.0
J_afm = -5.0         # Antiferromagnetic coupling
D_dmi = 1.0
K_ani = 0.8
B_ext = np.array([0.0, 0.0, 0.0])

J_sot = 0.3          # Driver Current
p_pol = np.array([0.0, 1.0, 0.0])

prefactor_prec = -gamma / (1 + alpha**2)
prefactor_damp = -(gamma * alpha) / (1 + alpha**2)

# ==========================================
# 2. Initialization & Core Dynamics
# ==========================================
def init_afm_skyrmion():
    m1 = np.zeros((Nx, Ny, 3))
    cx, cy = Nx // 5, Ny // 2  # Start on the left
    R_core = 6.0

    for i in range(Nx):
        for j in range(Ny):
            r_sq = (i - cx)**2 + (j - cy)**2
            theta = np.pi * np.exp(-r_sq / (R_core**2))
            phi = np.arctan2(j - cy, i - cx)

            m1[i, j, 0] = np.sin(theta) * np.cos(phi)
            m1[i, j, 1] = np.sin(theta) * np.sin(phi)
            m1[i, j, 2] = np.cos(theta)

    norms = np.linalg.norm(m1, axis=2, keepdims=True)
    m1 = m1 / np.where(norms == 0, 1, norms)
    return m1, -m1

def laplacian(m):
    return (np.roll(m, 1, axis=0) + np.roll(m, -1, axis=0) +
            np.roll(m, 1, axis=1) + np.roll(m, -1, axis=1) - 4 * m) / (dx**2)

def gradient(m):
    dm_dx = (np.roll(m, -1, axis=1) - np.roll(m, 1, axis=1)) / (2 * dx)
    dm_dy = (np.roll(m, -1, axis=0) - np.roll(m, 1, axis=0)) / (2 * dx)
    return dm_dx, dm_dy

def llgs_rhs_coupled(m1, m2):
    # Field for m1
    H1_ex = 2 * A_ex * laplacian(m1)
    H1_ani = np.zeros_like(m1); H1_ani[:,:,2] = 2 * K_ani * m1[:,:,2]
    dm1_dx, dm1_dy = gradient(m1)
    H1_dmi = np.zeros_like(m1)
    H1_dmi[:,:,0] = -2 * D_dmi * dm1_dx[:,:,2]
    H1_dmi[:,:,1] = -2 * D_dmi * dm1_dy[:,:,2]
    H1_dmi[:,:,2] = 2 * D_dmi * (dm1_dx[:,:,0] + dm1_dy[:,:,1])
    Heff1 = H1_ex + (J_afm * m2) + H1_ani + H1_dmi + B_ext

    # Field for m2
    H2_ex = 2 * A_ex * laplacian(m2)
    H2_ani = np.zeros_like(m2); H2_ani[:,:,2] = 2 * K_ani * m2[:,:,2]
    dm2_dx, dm2_dy = gradient(m2)
    H2_dmi = np.zeros_like(m2)
    H2_dmi[:,:,0] = -2 * D_dmi * dm2_dx[:,:,2]
    H2_dmi[:,:,1] = -2 * D_dmi * dm2_dy[:,:,2]
    H2_dmi[:,:,2] = 2 * D_dmi * (dm2_dx[:,:,0] + dm2_dy[:,:,1])
    Heff2 = H2_ex + (J_afm * m1) + H2_ani + H2_dmi + B_ext

    # SOT Torques
    p_field = np.ones_like(m1) * p_pol
    tau1 = J_sot * np.cross(m1, np.cross(m1, p_field))
    tau2 = J_sot * np.cross(m2, np.cross(m2, p_field))

    # Dynamics
    m1xH1 = np.cross(m1, Heff1)
    dm1_dt = (prefactor_prec * m1xH1) + (prefactor_damp * np.cross(m1, m1xH1)) + tau1
    m2xH2 = np.cross(m2, Heff2)
    dm2_dt = (prefactor_prec * m2xH2) + (prefactor_damp * np.cross(m2, m2xH2)) + tau2

    return dm1_dt, dm2_dt

def rk4_step_afm(m1, m2):
    k1_1, k1_2 = llgs_rhs_coupled(m1, m2)
    k2_1, k2_2 = llgs_rhs_coupled(m1 + 0.5*dt*k1_1, m2 + 0.5*dt*k1_2)
    k3_1, k3_2 = llgs_rhs_coupled(m1 + 0.5*dt*k2_1, m2 + 0.5*dt*k2_2)
    k4_1, k4_2 = llgs_rhs_coupled(m1 + dt*k3_1, m2 + dt*k3_2)

    m1_new = m1 + (dt / 6.0) * (k1_1 + 2*k2_1 + 2*k3_1 + k4_1)
    m2_new = m2 + (dt / 6.0) * (k1_2 + 2*k2_2 + 2*k3_2 + k4_2)

    m1_new /= np.linalg.norm(m1_new, axis=2, keepdims=True)
    m2_new /= np.linalg.norm(m2_new, axis=2, keepdims=True)
    return m1_new, m2_new

# ==========================================
# 3. 3D Plotting & Animation Setup
# ==========================================
m1, m2 = init_afm_skyrmion()

fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')
plt.close(fig) # Prevent duplicate inline plot

# Subsampling parameters (Plot every 2nd spin to save CPU)
stride = 2
X, Y = np.meshgrid(np.arange(0, Nx, stride), np.arange(0, Ny, stride), indexing='ij')
Z = np.zeros_like(X)

def draw_3d_frame():
    """Clears and redraws the 3D quiver plot (cones)."""
    ax.cla()

    # Calculate Neel vector L
    L = (m1 - m2) / 2.0

    # Subsample data
    u = L[::stride, ::stride, 0]
    v = L[::stride, ::stride, 1]
    w = L[::stride, ::stride, 2]

    # Color mapping based on the Z-component (w)
    colors = cm.coolwarm((w + 1) / 2.0)
    colors = colors.reshape(-1, 4) # Flatten for quiver

    # Draw cones
    q = ax.quiver(X, Y, Z, u, v, w,
                  length=0.3, normalize=True,
                  arrow_length_ratio=0.1, pivot='middle',
                  colors=colors)

    ax.set_zlim(-1.5, 1.5)
    ax.set_xlim(0, Nx)
    ax.set_ylim(0, Ny)
    ax.set_title("3D AFM Skyrmion (Néel Vector) moving on Racetrack")
    ax.set_axis_off() # Hide grid lines for cleaner look

def update(frame):
    global m1, m2
    # Run physics steps
    for _ in range(15): # Removed inner tqdm
        m1, m2 = rk4_step_afm(m1, m2)
    # Update visual frame
    draw_3d_frame()
    return fig,

# Generate animation
ani = FuncAnimation(fig, update, frames=tqdm(range(150), desc="Generating Animation"), interval=100, blit=False)
HTML(ani.to_jshtml())

# Comprehensive Micromagnetics: Simulating Skyrmion Dynamics

This notebook simulates the dynamics of magnetic spin textures using the continuum micromagnetic framework. Instead of modeling discrete atomic moments, we approximate the magnetic state as a continuous vector field: the normalized magnetization $\mathbf{m}(\mathbf{r}, t)$, where $|\mathbf{m}| = 1$.

## 1. The Energy Functional and Effective Field

The state of a magnetic system is governed by its free energy $E_\text{total}$. At any given time, the local spins experience an **Effective Magnetic Field** ($\mathbf{H}_\text{eff}$), defined as the negative variational derivative of the energy functional:

$$\mathbf{H}_\text{eff} = -\frac{1}{\mu_0 M_s} \frac{\delta E_\text{total}}{\delta \mathbf{m}}$$

Where $\mu_0$ is the vacuum permeability and $M_s$ is the saturation magnetization. Our numerical model accounts for five competing interactions:

1. **Exchange Interaction ($E_\text{ex}$):** Penalizes spatial variations, forcing neighboring spins to align (FM) or anti-align (AFM).
   $$\mathbf{H}_\text{ex} = \frac{2A}{\mu_0 M_s} \nabla^2 \mathbf{m}$$
2. **Dzyaloshinskii-Moriya Interaction ($E_\text{DMI}$):** An antisymmetric exchange preferring 90-degree spin orthogonalization, driving the chiral twisting required for skyrmions.
   $$\mathbf{H}_\text{DMI} = \frac{2D}{\mu_0 M_s} \left[ (\nabla \cdot \mathbf{m})\hat{z} - \nabla m_z \right]$$
3. **Magnetic Anisotropy ($E_\text{ani}$):** Perpendicular Magnetic Anisotropy (PMA) forces spins along an "easy axis" (the z-axis).
   $$\mathbf{H}_\text{ani} = \frac{2K}{\mu_0 M_s} m_z \hat{z}$$
4. **Zeeman Energy ($E_\text{zee}$):** Alignment with an external applied magnetic field.
   $$\mathbf{H}_\text{zee} = \mathbf{B}_\text{ext} / \mu_0$$
5. **Demagnetization / Dipole-Dipole ($E_\text{demag}$):** The long-range stray field generated by the magnetization itself. It acts to minimize surface and volume magnetic charges ($\nabla \cdot \mathbf{m}$).

---

## 2. Magnetization Dynamics: The LLGS Equation

The time evolution of $\mathbf{m}$ is described by the **Landau-Lifshitz-Gilbert-Slonczewski (LLGS)** equation. To solve this numerically using explicit time-integration (like 4th-Order Runge-Kutta), we use the explicit form:

$$\frac{\partial \mathbf{m}}{\partial t} = -\frac{\gamma}{1+\alpha^2} \mathbf{m} \times \mathbf{H}_\text{eff} - \frac{\gamma \alpha}{1+\alpha^2} \mathbf{m} \times (\mathbf{m} \times \mathbf{H}_\text{eff}) + \mathbf{\tau}_\text{driver}$$

Where $\gamma$ is the gyromagnetic ratio, $\alpha$ is the Gilbert damping, and $\mathbf{\tau}_\text{driver}$ is the Slonczewski spin-transfer torque: $\tau \propto \mathbf{m} \times (\mathbf{m} \times \mathbf{p})$, with $\mathbf{p}$ being the current polarization.

---

## 3. Topology and the Skyrmion Hall Effect (SkHE)



A skyrmion is a topologically protected quasiparticle, quantified by its topological charge or winding number $Q$:

$$Q = \frac{1}{4\pi} \iint \mathbf{m} \cdot \left( \frac{\partial \mathbf{m}}{\partial x} \times \frac{\partial \mathbf{m}}{\partial y} \right) dx dy$$

For a perfect skyrmion, $Q = \pm 1$. This topological charge directly dictates its motion. The rigid-body dynamics of a skyrmion are described mathematically by the **Thiele Equation**:

$$\mathbf{G} \times \mathbf{v} + \alpha \mathcal{D} \mathbf{v} = \mathbf{F}_\text{driver}$$

Here, $\mathbf{v}$ is the skyrmion velocity, $\mathcal{D}$ is the dissipation tensor, and $\mathbf{F}_\text{driver}$ is the force from the applied current. The crucial term is the **Gyrovector** $\mathbf{G} = (0, 0, -4\pi Q)$.

Because $Q \neq 0$ for a ferromagnet, the cross product $\mathbf{G} \times \mathbf{v}$ generates a fictitious **Magnus force** perpendicular to the velocity. This forces the skyrmion to move diagonally relative to the applied current—a phenomenon known as the **Skyrmion Hall Effect (SkHE)**.

---

## 4. Numerical Stability: The "Goldilocks Zone"

Simulating skyrmions is notoriously difficult due to extreme mathematical sensitivity. Skyrmions only exist within a narrow stability window defined by the dimensionless parameter $\kappa$:

$$\kappa = \frac{\pi D}{4 \sqrt{A K_\text{eff}}}$$

Where $K_\text{eff} = K - \frac{1}{2}\mu_0 M_s^2$.
* If **$\kappa \ll 1$ (Collapse):** The anisotropy or external Zeeman field is too strong, crushing the skyrmion until its radius becomes smaller than the grid size ($dx$). Once it shrinks below the material's exchange length $\Delta = \pi \sqrt{A / K_\text{eff}}$, the mathematical continuum breaks down, and the skyrmion falls through the lattice, vanishing ($Q \to 0$).
* If **$\kappa > 1$ (Explosion):** The DMI is too strong relative to the anisotropy, or the Zeeman field favors the core rather than the background. The skyrmion expands uncontrollably into a stripe-domain or spin-spiral phase.



---

## 5. Boundary Conditions and Demagnetization Limits

Calculating the Demagnetization field requires a computationally heavy convolution of a dipole tensor with the magnetization.

* **The Open Boundary Problem:** If a finite grid is zero-padded to calculate a linear FFT convolution, it simulates a magnetic sample abruptly ending in a vacuum. This creates massive magnetic surface charges ($\nabla \cdot \mathbf{m} \neq 0$) at the edges. These charges emit immense stray fields that propagate inward and violently crush the skyrmion.
* **The Periodic Boundary Solution:** By using a Circular FFT Convolution without zero-padding, we simulate an infinitely repeating magnetic sheet. This eliminates edge charges and stray boundary fields, mathematically stabilizing the skyrmion and allowing it to wrap seamlessly around the screen.

---

## 6. The Antiferromagnetic (AFM) Solution



To solve the problematic Skyrmion Hall Effect for memory applications, we simulate Antiferromagnetic (AFM) skyrmions. An AFM system consists of two closely coupled magnetic sublattices ($\mathbf{m}_1$ and $\mathbf{m}_2$) forced into anti-alignment by a massive negative exchange coupling:

$$E_\text{AFM} = -J_\text{AFM} (\mathbf{m}_1 \cdot \mathbf{m}_2)$$

Because $\mathbf{m}_2 = -\mathbf{m}_1$, the topological charges cancel perfectly ($Q_\text{total} = Q_1 + Q_2 = -1 + 1 = 0$).

**Mathematical Consequences:**
1. **No Hall Effect:** Looking at the Thiele equation, if $Q=0$, the Gyrovector $\mathbf{G} = 0$. The Magnus force disappears entirely, allowing the AFM skyrmion to move perfectly straight along the current path.
2. **Zero Demagnetization:** Because the macroscopic magnetization is zero ($\mathbf{m}_1 + \mathbf{m}_2 = 0$), there are no stray dipole fields. The highly complex FFT demagnetization calculations are unnecessary.
3. **Terahertz Dynamics:** The massive restoring force of $J_\text{AFM}$ acts like a highly stiff mathematical spring. While this makes the AFM skyrmion physically robust against collapsing, it requires the numerical time step ($dt$) to be reduced by an order of magnitude to prevent Runge-Kutta solver overshoots.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from tqdm.notebook import tqdm # Import tqdm for progress bar

# Increase animation embedding limit to accommodate larger animations
plt.rcParams['animation.embed_limit'] = 100.0 # Increase to 100 MB

# ==========================================
# 1. Physics Parameters (Stable PBC Racetrack)
# ==========================================
Nx, Ny = 64, 64      # Powers of 2 make the FFT significantly faster
dx = 1.0             # Spatial step
dt = 0.01            # Small time step for RK4 stability
alpha = 0.3          # Gilbert damping
gamma = 1.0          # Gyromagnetic ratio

# The "Goldilocks" Interaction Parameters
A_ex = 1.0           # Exchange stiffness
D_dmi = 1.2          # DMI strength (High enough to twist the spins)
K_ani = 1.0          # Perpendicular Anisotropy
B_ext = np.array([0.0, 0.0, 0.1]) # Gentle stabilizing Zeeman field
Ms = 0.05            # Saturation Magnetization (controls Demag strength)

# Spin Transfer Torque (STT) Driver
J_stt = 0.15         # Driving current strength
p_pol = np.array([1.0, 0.0, 0.0]) # Current pushing in +X direction

# Precompute LLGS constants
prefactor_prec = -gamma / (1 + alpha**2)
prefactor_damp = -(gamma * alpha) / (1 + alpha**2)

# ==========================================
# 2. PBC Demagnetization Kernel (Circular FFT)
# ==========================================
def build_pbc_demag_kernel():
    """Builds a discrete dipole tensor mapped for periodic boundary conditions."""
    K = np.zeros((Nx, Ny, 3, 3))

    # Create wrapped coordinate grid for shortest-path distances
    rx = np.arange(Nx, dtype=float)
    rx[rx > Nx // 2] -= Nx
    ry = np.arange(Ny, dtype=float)
    ry[ry > Ny // 2] -= Ny
    rx, ry = np.meshgrid(rx, ry, indexing='ij')

    r2 = rx**2 + ry**2
    r2[0, 0] = 1.0  # Prevent divide-by-zero at the origin
    r5 = r2**(2.5)

    # In-plane stray fields (Dipole approximation)
    K[:, :, 0, 0] = (3 * rx**2 - r2) / r5
    K[:, :, 1, 1] = (3 * ry**2 - r2) / r5
    K[:, :, 0, 1] = (3 * rx * ry) / r5
    K[:, :, 1, 0] = K[:, :, 0, 1]

    # Out-of-plane self-demagnetization (Thin film limit)
    K[0, 0, :, :] = 0
    K[0, 0, 2, 2] = -4 * np.pi

    # Since it's already centered at [0,0] with wrapped distances, we just FFT it
    return np.fft.fftn(K, axes=(0, 1))

# Precompute this massively heavy tensor once!
FFT_KERNEL_PBC = build_pbc_demag_kernel()

def calc_demag_field_pbc(m):
    """Calculates stray field using Circular Convolution (NO zero-padding)."""
    m_fft = np.fft.fftn(m, axes=(0, 1))

    # Einstein summation for ultra-fast Tensor * Vector multiplication in k-space
    H_demag_fft = np.einsum('xyij,xyj->xyi', FFT_KERNEL_PBC, m_fft)

    # Inverse FFT back to real space
    H_demag = np.fft.ifftn(H_demag_fft, axes=(0, 1)).real
    return H_demag * Ms

# ==========================================
# 3. PBC Spatial Derivatives & LLGS Dynamics
# ==========================================
def get_derivatives_pbc(m):
    """Calculates spatial derivatives wrapping around the grid edges."""
    laplacian = (np.roll(m, 1, axis=0) + np.roll(m, -1, axis=0) +
                 np.roll(m, 1, axis=1) + np.roll(m, -1, axis=1) - 4 * m) / (dx**2)

    dm_dx = (np.roll(m, -1, axis=0) - np.roll(m, 1, axis=0)) / (2 * dx)
    dm_dy = (np.roll(m, -1, axis=1) - np.roll(m, 1, axis=1)) / (2 * dx)

    return laplacian, dm_dx, dm_dy

def llgs_rhs(m):
    laplacian, dm_dx, dm_dy = get_derivatives_pbc(m)

    # 1. Exchange
    H_ex = 2 * A_ex * laplacian

    # 2. Anisotropy
    H_ani = np.zeros_like(m)
    H_ani[:, :, 2] = 2 * K_ani * m[:, :, 2]

    # 3. DMI
    H_dmi = np.zeros_like(m)
    H_dmi[:, :, 0] = -2 * D_dmi * dm_dx[:, :, 2]
    H_dmi[:, :, 1] = -2 * D_dmi * dm_dy[:, :, 2]
    H_dmi[:, :, 2] = 2 * D_dmi * (dm_dx[:, :, 0] + dm_dy[:, :, 1])

    # 4. Demagnetization & Zeeman
    H_demag = calc_demag_field_pbc(m)
    Heff = H_ex + H_ani + H_dmi + H_demag + B_ext

    # 5. Spin Transfer Torque
    p_field = np.ones_like(m) * p_pol
    tau_stt = J_stt * np.cross(m, np.cross(m, p_field))

    # Combine into LLGS
    m_cross_H = np.cross(m, Heff)
    return (prefactor_prec * m_cross_H) + (prefactor_damp * np.cross(m, m_cross_H)) + tau_stt

def rk4_step(m):
    """4th-Order Runge-Kutta Integration"""
    k1 = llgs_rhs(m)
    k2 = llgs_rhs(m + 0.5 * dt * k1)
    k3 = llgs_rhs(m + 0.5 * dt * k2)
    k4 = llgs_rhs(m + dt * k3)

    m_new = m + (dt / 6.0) * (k1 + 2*k2 + 2*k3 + k4)
    # Always re-normalize to maintain |m| = 1
    return m_new / np.linalg.norm(m_new, axis=2, keepdims=True)

# ==========================================
# 4. Initialization and Jupyter Animation
# ==========================================
def init_skyrmion_smooth():
    """Initializes a skyrmion with a smooth Gaussian profile."""
    m = np.zeros((Nx, Ny, 3))
    cx, cy = Nx // 2, Ny // 2  # Start exactly in the center
    R_core = 7.0  # Generous radius for survival

    for i in range(Nx):
        for j in range(Ny):
            r_sq = (i - cx)**2 + (j - cy)**2
            theta = np.pi * np.exp(-r_sq / (R_core**2))
            phi = np.arctan2(j - cy, i - cx)
            m[i, j, 0] = np.sin(theta) * np.cos(phi)
            m[i, j, 1] = np.sin(theta) * np.sin(phi)
            m[i, j, 2] = np.cos(theta)

    return m / np.linalg.norm(m, axis=2, keepdims=True)

m = init_skyrmion_smooth()

# Plot Setup
fig, ax = plt.subplots(figsize=(6, 6))
plt.close(fig) # Prevent duplicate static output

cax = ax.imshow(m[:, :, 2].T, cmap='coolwarm', vmin=-1, vmax=1, origin='lower')
ax.set_title("FM Skyrmion (PBC) - Skyrmion Hall Effect")
fig.colorbar(cax, label="Mz (Out-of-Plane)")

def update(frame):
    global m
    for _ in range(10): # Run 10 physics iterations per visual frame
        m = rk4_step(m)

    cax.set_array(m[:, :, 2].T)
    return cax,

# Generate HTML5 Video for Jupyter
ani = FuncAnimation(fig, update, frames=tqdm(range(1000), desc="Generating Animation"), interval=50, blit=True)
HTML(ani.to_jshtml())